In [ ]:
#pip install keras-tuner --upgrade

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
%matplotlib inline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout

In [ ]:
def fetch_dataset(path):
    images = []
    labels = []
    for dir1 in os.listdir(path):
        for file in os.listdir(os.path.join(path,dir1)):
            image_path = os.path.join(path,dir1,file)
            with Image.open(image_path).convert('L') as img:
                arr = np.array(img.getdata()).reshape(256, 256, 1) / 255
                images.append(arr)
                labels.append(dir1)
    
    unique_labels = {k: v for v,k in enumerate(np.unique(labels))}
    numeric_labels = [unique_labels[labels[i]] for i in range(len(labels))]
    #print(numeric_labels)
    return np.array(images),numeric_labels


In [ ]:
train_path = os.path.join("data","train")
val_path = os.path.join("data","val")
test_path = os.path.join("data","test")


x_train,y_train = fetch_dataset(train_path)
x_val,y_val = fetch_dataset(val_path)
#x_test,y_test = fetch_dataset(test_path)

In [ ]:
y_train_classes = y_train
y_val_classes = y_val
y_train = tf.keras.utils.to_categorical(y_train_classes, 4)
y_val = tf.keras.utils.to_categorical(y_val_classes, 4)

In [ ]:
classes = ["COVID", "HEALTHY", "PNEUMONIA_BAC", "PNEUMONIA_VIR"]

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# early stopping values
callback_stop = tf.keras.callbacks.EarlyStopping(
    monitor="recall_m",
    min_delta=0.0003,
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True)

# Keras Tuner part

### Iteration 1

Amount of convolutions/feature maps per layer

In [ ]:
import keras_tuner as kt
from keras_tuner import Objective

In [ ]:
# def build_model_1():
#     data_dims = (256, 256, 1)

#     model = Sequential()
#     model.add(Conv2D(20,kernel_size=(3, 3), activation="relu", padding="same", input_shape=data_dims))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Conv2D(20, kernel_size=(3, 3), activation="relu", padding="same"))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.30))
#     model.add(Conv2D(20, kernel_size=(3, 3), activation="relu", padding="same"))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Conv2D(20, kernel_size=(3, 3), activation="relu", padding="same"))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.30))
#     model.add(Flatten())
#     model.add(Dense(16, activation="sigmoid"))
#     model.add(Dropout(0.25))
#     model.add(Dense(4, activation="softmax"))

#     model.compile(
#         optimizer="adam",
#         loss="categorical_crossentropy",
#         metrics=["accuracy", "categorical_crossentropy", f1_m, precision_m, recall_m])
#     return model

In [ ]:
# model = build_model_1()

# model.summary()

In [ ]:
# model.fit(
#     x_train.reshape(-1, 256, 256, 1), y_train,
#     epochs=5,
#     batch_size=512,
#     validation_data=(x_val.reshape(-1, 256, 256, 1), y_val),
#     verbose=1
#     )

In [ ]:
def build_model_2(hp):
    data_dims = (256, 256, 1)
    units = hp.Int("units", min_value=5, max_value=50, step=5)

    model = Sequential()
    model.add(Conv2D(units,kernel_size=(3, 3), activation="relu", padding="same", input_shape=data_dims))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(units, kernel_size=(3, 3), activation="relu", padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.30))
    model.add(Conv2D(units, kernel_size=(3, 3), activation="relu", padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(units, kernel_size=(3, 3), activation="relu", padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.30))
    model.add(Flatten())
    model.add(Dense(16, activation="sigmoid"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="softmax"))

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", "categorical_crossentropy", f1_m, precision_m, recall_m])
    return model

In [ ]:
build_model_2(kt.HyperParameters())


In [ ]:
tuner = kt.RandomSearch(
    hypermodel=build_model_2,
    objective=Objective("val_recall_m", direction="max"),
    max_trials=20,
    executions_per_trial=1,
    overwrite=True,
    directory="my_dir",
    project_name="x-ray-test"
)

tuner.search(x_train, y_train, 
             epochs=3, 
             batch_size=512,
             validation_data=(x_val, y_val), 
             verbose=1
        )


In [ ]:
# Get the top 3 models.
models = tuner.get_best_models(num_models=3)
best_model = models[0]
second_model = models[1]
third_model = models[2]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 256, 256))
best_model.summary()
second_model.build(input_shape=(None, 256, 256))
second_model.summary()
third_model.build(input_shape=(None, 256, 256))
third_model.summary()

In [ ]:
tuner.results_summary()